#Previsione Livelli Ricoveri Terapia Intensiva, Nuovi Positivi e Deceduti per COVID

Nel seguente notebook sono stati utilizzati tre diversi modelli, LSTM, XGBoost e ARIMA con i dati resi disponibili dal Ministero della Salute per effettuare un forecast con orizzonte 1, 2, 7 e 14 giorni dei valori considerati.

Sono stati usati i dati regionali resi disponibili via file .csv al [link](https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv) e filtrati per regione di interesse, in questo caso Emilia-Romagna.
Ogni riga è un aggiornamento giornaliero dei seguenti valori: 
data, stato, codice_regione, denominazione_regione, lat, long, ricoverati_con_sintomi, terapia_intensiva,totale_ospedalizzati, isolamento_domiciliare, totale_positivi, variazione_totale_positivi, nuovi_positivi, dimessi_guariti, deceduti, casi_da_sospetto_diagnostico, casi_da_screening, totale_casi, tamponi, casi_testati, note, ingressi_terapia_intensiva, note_test, note_casi, totale_positivi_test_molecolare, totale_positivi_test_antigenico_rapido, tamponi_test_molecolare, tamponi_test_antigenico_rapido, codice_nuts_1, codice_nuts_2. 

Di questi sono state poi utilizzate per l'analisi: 
data, ricoverati_con_sintomi, terapia_intensiva, totale_ospedalizzati, variazione_totale_positivi, nuovi_positivi, deceduti, tamponi, ingressi_terapia_intensiva.

I valori di deceduti e tamponi sono stati differenziati per avere l'incremento giornaliero invece del dato cumulativo che era presente.

L'analisi svolta è su singola variabile.

I dati sono stati divisi con percentuale di 80 e 20 rispettivamente per training e testing.

Il primo 80% di dati in fase di tuning è a sua volta divisa in 80% di train e 20% di validation, in modo che il tuning non venga effettuato su dati di test.

Dopo la suddivisione è stato effettuato lo scaling attravero un MinMaxScaler che comprime i valori tra -1 ed 1 (la trasformazione viene poi invertita dopo la previsione).

Per i modelli XGBoost e LSTM sono stati preparati i dati creando per ogni giorno un array di lag temporali e di valori futuri della variabile presa in considerazione:
(t-n,...,t-1,t) -> (t+1,...,t+p) con n = giorni di lag e p = orizzonte di previsione.
Questi valori sono stati utilizzati per il training dei due modelli utilizzando le variabili lag come ingresso del modello e i valori futuri come target.

L'errore considerato è il Mean Absolute Error sull'orizzonte di previsione. Viene poi considerata la media dei MAE calcolati su tutte le previsioni.

Gli iper-parametri sono stati scelti attraverso una grid search per ogni variabile e per ogni orizzonte temporale selezionando il modello che da una media dei MAE inferiore.

Il grafico mostra lo slot temporale di previsione con MAE inferiore.


In [1]:
import os

if 'google.colab' in str(get_ipython()):
    files = ['util.py',
             'windows.py',
             'models']

    for file in files:
        os.system('rm ./' + file)
        os.system(
            'wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/' + file)

import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from numpy.random import seed
import plotly.graph_objects as go

# fix for 'package not found' when installing in Anaconda environment
if 'google.colab' not in str(get_ipython()):
    import pip
    pip.main(['install', 'xgboost'])

from xgboost import XGBRegressor
from util import select_relevant_rows, select_attributes, read_movement_data, download_updated_mobility_data, download_updated_mobility_data, save_config, load_config


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [10]:
use_existing_config = False
column_to_predict = 'terapia_intensiva'
columns = ['nuovi_positivi', 'terapia_intensiva', 'deceduti']
split_percent = 0.80
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'
n_futures = [1, 2, 7, 14]

In [3]:
local_region_path = r'../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'
remote_region_path = r'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

regions_frame = pd.read_csv(remote_region_path)

region_focus_data = select_relevant_rows(
    regions_frame,
    attribute_focus,
    region_focus
)

frame_interesting_columns = select_attributes(region_focus_data, [
    'data',
    'ricoverati_con_sintomi',
    'terapia_intensiva',
    'totale_ospedalizzati',
    'variazione_totale_positivi',
    'nuovi_positivi',
    'deceduti',
    'tamponi',
    'ingressi_terapia_intensiva'
])

frame_interesting_columns = pd.DataFrame(frame_interesting_columns)
frame_interesting_columns['data'] = pd.to_datetime(
    frame_interesting_columns['data'])
frame_interesting_columns['data'] = frame_interesting_columns['data'].dt.strftime(
    r'%Y-%m-%d')
frame_interesting_columns['data'] = pd.to_datetime(frame_interesting_columns['data'])
frame_interesting_columns = frame_interesting_columns[frame_interesting_columns['data'] < pd.to_datetime('2022-01-31')]
frame_interesting_columns = frame_interesting_columns.fillna(0)

frame_interesting_columns.rename(columns={'data': 'date'}, inplace=True)
frame_interesting_columns.set_index('date', inplace=True)

# revert cumulative data
frame_interesting_columns['deceduti'] = frame_interesting_columns['deceduti'].diff(
)
frame_interesting_columns['tamponi'] = frame_interesting_columns['tamponi'].diff(
)
frame_interesting_columns.dropna(inplace=True)

frame_interesting_columns.tail

# numpy seed
seed(1)

In [4]:
for column in frame_interesting_columns.columns:
    frame = frame_interesting_columns[column]
    trace = go.Scatter(
        x=frame.index, y=frame, mode='lines', name='Prediction')
    layout = go.Layout(
        title=column, xaxis={'title': 'Date'},
        yaxis={'title': column})
    fig = go.Figure(
        data=[trace], layout=layout)
    fig.show()

In [5]:
def split_series(series, n_past, n_future, arima=False):
    X, y, X_indexes, y_indexes = list(), list(), list(), list()
    index = np.array(series.index).reshape(series.values.shape[0], 1)
    series = series.values

    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        start = 0 if arima == True else window_start

        past, future = series[start:past_end,
                              :], series[past_end:future_end, :]
        past_index, future_index = index[start:past_end,
                                         :], index[past_end:future_end, :]
        X.append(past)
        y.append(future)
        X_indexes.append(past_index)
        y_indexes.append(future_index)

    return np.array(X), np.array(y), np.array(X_indexes), np.array(y_indexes)


def plot_best_pred(
    sorted_results, column_to_predict):
        pred = sorted_results[0][1][-1]['pred']
        test = sorted_results[0][1][-1]['y_test']

        prediction_trace = go.Scatter(
            x=pred.index, y=pred, mode='lines', name='Prediction')
        truth_trace = go.Scatter(
            x=test.index, y=test, mode='lines', name='Ground Truth')
        layout = go.Layout(
            title=column_to_predict, xaxis={'title': 'Date'},
            yaxis={'title': column_to_predict})
        fig = go.Figure(
            data=[prediction_trace, truth_trace], layout=layout)
        fig.show()

#LSTM Multi Output 

In [6]:
def define_compile_lstm(config, input_shape, n_future=7):
    model = Sequential()
    model.add(LSTM(config['input'], activation=config['activation'],
                   input_shape=input_shape, return_sequences=True))
    model.add(LSTM(config['hidden'], activation=config['activation'],
                   return_sequences=False))
    model.add(Dropout(config['dropout']))
    model.add(Dense(n_future))
    model.compile(optimizer=config['optimizer'], loss=config['loss'])
    return model


def define_lstm_configs():
    input = [32, 64, 128]
    hidden = [32, 64, 128]
    activation = ['relu']
    dropout = [0.1, 0.2]
    out = [1]
    optimizer = ['adam']
    loss = ['mae']
    look_back = [28]

    configs = []
    keys = ['input', 'hidden', 'activation', 'dropout', 'out',
            'optimizer', 'loss', 'look_back']

    for i in input:
        for j in hidden:
            for k in activation:
                for l in dropout:
                    for m in out:
                        for n in optimizer:
                            for o in loss:
                                for p in look_back:
                                        config = dict(
                                            zip(keys, (i, j, k, l, m, n, o, p)))
                                        configs.append(config)

    return configs


def execute_lstm(
        dataframe, column_to_predict, config, split_percent=0.80, multi=False, n_future=7):
    df = dataframe[column_to_predict].copy()

    split = int(split_percent*len(df))

    n_past = config['look_back']
    n_features = 1

    train, test = pd.DataFrame(df[:split]), pd.DataFrame(df[split:])

    scalers = {}

    for i in train.columns:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        s_s = scaler.fit_transform(train[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_' + i] = scaler
        train[i] = s_s

    for i in test.columns:
        scaler = scalers['scaler_'+i]
        s_s = scaler.transform(test[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_'+i] = scaler
        test[i] = s_s

    X_train, y_train, X_train_indexes, y_train_indexes = split_series(
        train, n_past, n_future)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))

    X_test, y_test, X_test_indexes, y_test_indexes = split_series(
        test, n_past, n_future)
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

    model = define_compile_lstm(config, input_shape=(n_past,n_features), n_future=n_future)

    history = model.fit(
        X_train, y_train, epochs=50, validation_data=(X_test, y_test),
        verbose=0, shuffle=False)

    pred = model.predict(X_test)

    pred = pred.reshape(pred.shape[0], pred.shape[1], 1)

    for index, i in enumerate(train.columns):
        scaler = scalers['scaler_'+i]

        pred[:, :, index] = scaler.inverse_transform(pred[:, :, index])

        y_train[:, :, index] = scaler.inverse_transform(y_train[:, :, index])
        y_test[:, :, index] = scaler.inverse_transform(y_test[:, :, index])

    df_results = []

    for i in range(len(y_test)):
        current = pd.DataFrame(
            {'y_test':y_test[i].reshape(-1),
            'pred':pred[i].reshape(-1),
            'dates':y_test_indexes[i].reshape(-1)})
        current.set_index('dates', inplace=True)
        df_results.append(current)

    results = {}

    for el in df_results:
        mae = mean_absolute_error(el['y_test'], el['pred'])
        mape = mean_absolute_percentage_error(el['y_test'], el['pred'])
        results[el.index[0]] = (mae, mape, el)

    sorted_results = sorted(results.items(), key=lambda x: x[1][0])

    return sorted_results


def grid_search_lstm(
        dataframe, column_to_predict, split_percent=0.80, multi=False, n_future=7):
    configs = define_lstm_configs()
    results = []

    split = int(split_percent*len(dataframe))
    df_val = dataframe[:split].copy()

    for config in configs:
        sorted_results = execute_lstm(
            df_val, column_to_predict, config, split_percent=split_percent,
            multi=multi, n_future=n_future)

        results.append((np.mean(np.array(list(map(lambda x:x[1][0], sorted_results)))), config))

    return results

summaries = []

for n_future in n_futures:
    for column_to_predict in columns:
        config_path = region_focus + '_' + 'uni_lstm_config' + '_' + column_to_predict
        config_path = config_path + '_' + str(n_future)
        if use_existing_config:
            if not os.path.isfile(config_path):
                os.system('wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/configs/' + config_path)
            config = load_config(config_path)
            sorted_results = execute_lstm(
                frame_interesting_columns, column_to_predict=column_to_predict,
                config=config, split_percent=split_percent, multi=False, n_future=n_future)
            os.system('rm ' + config_path)
        else:
            results = grid_search_lstm(
                frame_interesting_columns, column_to_predict, split_percent=split_percent,
                multi=False, n_future=n_future)

            results.sort()
            config = results[0][-1]

            save_config(config_path, config)

            sorted_results = execute_lstm(
                frame_interesting_columns, column_to_predict=column_to_predict,
                config=config, split_percent=split_percent, multi=False, n_future=n_future)
                    
        avg_mae = np.mean(np.array(list(map(lambda x:x[1][0], sorted_results))))
        avg_mape = np.mean(np.array(list(map(lambda x:x[1][1], sorted_results))))

        summary = '|' + column_to_predict + '| seq len ' + str(n_future) + '| mae: ' + str(avg_mae) + '| mape: ' + str(avg_mape) + '| best config: ' + str(config)

        print(summary)
        summaries.append(summary)

        if n_future > 2:
            plot_best_pred(sorted_results, column_to_predict)
        else:
            print(('Pred: ', sorted_results[0][1][-1]['pred'].values))
            print(('Test: ', sorted_results[0][1][-1]['y_test'].values))

print(summaries)


|nuovi_positivi| seq len 1| mae: 33535.12620116117| mape: 2.2933874904622353| best config: {'input': 128, 'hidden': 128, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}
('Pred: ', array([376.88568], dtype=float32))
('Test: ', array([372.]))
|terapia_intensiva| seq len 1| mae: 7.810227611608673| mape: 0.16893385701442407| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}
('Pred: ', array([69.0928], dtype=float32))
('Test: ', array([69.]))
|deceduti| seq len 1| mae: 3.914289687809191| mape: 487404247808323.8| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}
('Pred: ', array([6.0376654], dtype=float32))
('Test: ', array([6.]))
|nuovi_positivi| seq len 2| mae: 82650.99085762227| mape: 5.302409849160477| best config: {'input': 64, 'hidden': 128, 'activati

|terapia_intensiva| seq len 7| mae: 9.706267278661171| mape: 0.13094981495325372| best config: {'input': 32, 'hidden': 32, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


|deceduti| seq len 7| mae: 4.663506347666343| mape: 492530436594541.94| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


|nuovi_positivi| seq len 14| mae: 5429.905428248381| mape: 0.5173949537147431| best config: {'input': 64, 'hidden': 64, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


|terapia_intensiva| seq len 14| mae: 15.21778547510905| mape: 0.2008632039489096| best config: {'input': 64, 'hidden': 32, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


|deceduti| seq len 14| mae: 4.65619834730615| mape: 544854214354761.8| best config: {'input': 32, 'hidden': 32, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


["|nuovi_positivi| seq len 1| mae: 33535.12620116117| mape: 2.2933874904622353| best config: {'input': 128, 'hidden': 128, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}", "|terapia_intensiva| seq len 1| mae: 7.810227611608673| mape: 0.16893385701442407| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}", "|deceduti| seq len 1| mae: 3.914289687809191| mape: 487404247808323.8| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}", "|nuovi_positivi| seq len 2| mae: 82650.99085762227| mape: 5.302409849160477| best config: {'input': 64, 'hidden': 128, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}", "|terapia_intensiva| seq len 2| mae: 5.068742245699451| mape: 0.0697478496081249| best config: {'input': 1

#XGBoost Multi Output

In [7]:
def define_xgb_configs():
    max_depth = [3, 6, 10]
    learning_rate = [0.01, 0.05, 0.1]
    n_estimators = [50, 100, 500, 1000]
    colsample_bytree = [0.3, 0.7]
    look_back = [28]
    objectives = ['reg:squarederror']

    configs = []
    keys = ['max_depth', 'learning_rate', 'n_estimators', 'colsample_bytree',
            'look_back', 'objective']

    for i in max_depth:
        for j in learning_rate:
            for k in n_estimators:
                for l in colsample_bytree:
                    for m in look_back:
                            for n in objectives:
                                config = dict(
                                    zip(keys, (i, j, k, l, m, n)))
                            configs.append(config)

    return configs


def execute_xgb(
        dataframe, column_to_predict, config, split_percent=0.80, n_future=7, multi=False):
    df = dataframe[column_to_predict].copy()
    split = int(split_percent*len(df))

    n_past = config['look_back']
    n_features = 1

    train, test = pd.DataFrame(df[:split]), pd.DataFrame(df[split:])

    scalers = {}

    for i in train.columns:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        s_s = scaler.fit_transform(train[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_' + i] = scaler
        train[i] = s_s

    for i in test.columns:
        scaler = scalers['scaler_'+i]
        s_s = scaler.transform(test[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_'+i] = scaler
        test[i] = s_s

    X_train, y_train, X_train_indexes, y_train_indexes = split_series(
        train, n_past, n_future)

    X_test, y_test, X_test_indexes, y_test_indexes = split_series(
        test, n_past, n_future)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1])
    X_train_indexes = X_train_indexes.reshape(
        X_train_indexes.shape[0], X_train_indexes.shape[1])
    y_train_indexes = y_train_indexes.reshape(
        y_train_indexes.shape[0], y_train_indexes.shape[1])

    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1])
    y_test = y_test.reshape(y_test.shape[0], y_test.shape[1])
    X_test_indexes = X_test_indexes.reshape(
        X_test_indexes.shape[0], X_test_indexes.shape[1])
    y_test_indexes = y_test_indexes.reshape(
        y_test_indexes.shape[0], y_test_indexes.shape[1])

    model = define_xgb(config)
    warnings.filterwarnings(action='ignore', category=UserWarning)
    model.fit(X_train, y_train)

    prediction = model.predict(X_test)  

    prediction[:, :] = scaler.inverse_transform(prediction[:, :])
    y_test[:, :] = scaler.inverse_transform(y_test[:, :])

    df_results = []

    for i in range(len(y_test)):
        current = pd.DataFrame(
            {'y_test':y_test[i].reshape(-1),
            'pred':prediction[i].reshape(-1),
            'dates':y_test_indexes[i].reshape(-1)})
        current.set_index('dates', inplace=True)
        df_results.append(current)

    results = {}

    for el in df_results:
        mae = mean_absolute_error(el['y_test'], el['pred'])
        mape = mean_absolute_percentage_error(el['y_test'], el['pred'])
        results[el.index[0]] = (mae, mape, el)

    sorted_results = sorted(results.items(), key=lambda x: x[1][0])

    return sorted_results


def grid_search_xgb(
        dataframe, column_to_predict, split_percent=0.80, n_future=7, multi=False):
    configs = define_xgb_configs()
    results = []

    split = int(split_percent*len(dataframe))
    df_val = dataframe[:split].copy()

    for config in configs:
        sorted_results = execute_xgb(
            df_val, column_to_predict, config, split_percent=split_percent,
            multi=multi, n_future=n_future)

        results.append((np.mean(np.array(list(map(lambda x:x[1][0], sorted_results)))), config))

    return results


def define_xgb(config):
    return MultiOutputRegressor(XGBRegressor(**config, verbosity=0))

summaries = []

for n_future in n_futures:
    for column_to_predict in columns:
        config_path = region_focus + '_' + 'uni_xgb_config' + '_' + column_to_predict
        config_path = config_path + '_' + str(n_future)
        if use_existing_config:
            if not os.path.isfile(config_path):
                os.system('wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/configs/' + config_path)
            config = load_config(config_path)
            sorted_results = execute_xgb(
                frame_interesting_columns, split_percent=split_percent, config=config,
                column_to_predict=column_to_predict, multi=False, n_future=n_future)
            os.system('rm ' + config_path)
        else:
            results = grid_search_xgb(
                frame_interesting_columns, split_percent=split_percent,
                column_to_predict=column_to_predict, multi=False)

            results.sort(key=lambda tup: tup[0])

            config = results[0][-1]

            save_config(config_path, config)

            sorted_results = execute_xgb(
                frame_interesting_columns, split_percent=split_percent, config=config,
                column_to_predict=column_to_predict, multi=False, n_future=n_future)

        avg_mae = np.mean(np.array(list(map(lambda x:x[1][0], sorted_results))))
        avg_mape = np.mean(np.array(list(map(lambda x:x[1][1], sorted_results))))

        summary = '|' + column_to_predict + '| seq len ' + str(n_future) + '| mae: ' + str(avg_mae) + '| mape: ' + str(avg_mape) + '| best config: ' + str(config)

        print(summary)
        summaries.append(summary)

        if n_future > 2:
            plot_best_pred(sorted_results, column_to_predict)
        else:
            print(('Pred: ', sorted_results[0][1][-1]['pred'].values))
            print(('Test: ', sorted_results[0][1][-1]['y_test'].values))

print(summaries)

|nuovi_positivi| seq len 1| mae: 3932.411760296738| mape: 0.3233823352952119| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}
('Pred: ', array([242.778], dtype=float32))
('Test: ', array([244.]))
|terapia_intensiva| seq len 1| mae: 6.871246622319805| mape: 0.0861392102709087| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 1000, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}
('Pred: ', array([140.06422], dtype=float32))
('Test: ', array([140.]))
|deceduti| seq len 1| mae: 4.037166256653635| mape: 358736163844455.6| best config: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 50, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}
('Pred: ', array([6.1258717], dtype=float32))
('Test: ', array([6.]))
|nuovi_positivi| seq len 2| mae: 3961.8003083625727| mape: 0.33769776914653193| best config: {'max_depth': 3, 'l

|terapia_intensiva| seq len 7| mae: 10.489884000606635| mape: 0.14349312618562637| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 1000, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}


|deceduti| seq len 7| mae: 4.471569883129584| mape: 428268209880481.56| best config: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 50, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}


|nuovi_positivi| seq len 14| mae: 3740.420193220298| mape: 0.389154299447662| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}


|terapia_intensiva| seq len 14| mae: 15.389784792693701| mape: 0.2083406000331712| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 1000, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}


|deceduti| seq len 14| mae: 5.1020234298840945| mape: 511088879413297.7| best config: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 50, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}


["|nuovi_positivi| seq len 1| mae: 3932.411760296738| mape: 0.3233823352952119| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}", "|terapia_intensiva| seq len 1| mae: 6.871246622319805| mape: 0.0861392102709087| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 1000, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}", "|deceduti| seq len 1| mae: 4.037166256653635| mape: 358736163844455.6| best config: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 50, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}", "|nuovi_positivi| seq len 2| mae: 3961.8003083625727| mape: 0.33769776914653193| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}", "|terapia_intensiva| seq len 2| mae: 7.325158203597615| mape: 0.09344950149070193|

#ARIMA Multi Output

In [14]:
def execute_arima(dataframe, order, column_to_predict, split_percent, n_future=7):
    df = dataframe[column_to_predict].copy()
    split = int(split_percent*len(df))

    n_past = 14
    n_features = 1

    train, test = pd.DataFrame(df[:split]), pd.DataFrame(df[split:])

    scalers = {}

    for i in train.columns:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        s_s = scaler.fit_transform(train[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_' + i] = scaler
        train[i] = s_s

    for i in test.columns:
        scaler = scalers['scaler_'+i]
        s_s = scaler.transform(test[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_'+i] = scaler
        test[i] = s_s

    X_test, y_test, X_test_indexes, y_test_indexes = split_series(
        test, n_past, n_future, arima=True)
    
    df_results = []

    history = [x for x in train.values]

    for i in range(len(X_test)):
        current_history = np.array(history).reshape(-1)
        current_history = np.append(current_history, X_test[i].reshape(-1))

        model = ARIMA(current_history, order=order)
        model_fitted = model.fit()
        prediction = model_fitted.forecast(n_future)[0]

        prediction = scaler.inverse_transform(prediction.reshape(-1,1))
        y_test[i] = scaler.inverse_transform(y_test[i])

        current = pd.DataFrame(
            {'y_test':y_test[i].reshape(-1),
            'pred':prediction.reshape(-1),
            'dates':y_test_indexes[i].reshape(-1)})

        current.set_index('dates', inplace=True)
        
        df_results.append(current)

    results = {}

    for el in df_results:
        mae = mean_absolute_error(el['y_test'], el['pred'])
        mape = mean_absolute_percentage_error(el['y_test'], el['pred'])
        results[el.index[0]] = (mae, mape, el)

    sorted_results = sorted(results.items(), key=lambda x: x[1][0])

    return sorted_results


def define_arima_configs():
    p_values = [10]
    d_values = [1]
    q_values = [1]
    return p_values, d_values, q_values


def evaluate_models(dataframe, column_to_predict, split_percent, n_future=7):
    p_values, d_values, q_values = define_arima_configs()
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                sorted_results = execute_arima(
                    dataframe, order, column_to_predict, split_percent, n_future)
                avg_error = np.mean(
                    np.array(list(map(lambda x:x[1][0], sorted_results))))
                if avg_error < best_score:
                    best_score, best_cfg = avg_error, order
    return best_cfg

summaries = []

for n_future in n_futures:
    for column_to_predict in columns:
        config_path = region_focus + '_' + 'arima_config' + '_' + column_to_predict
        config_path = config_path + '_' + str(n_future)
        if use_existing_config:
            if not os.path.isfile(config_path):
                os.system('wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/configs/' + config_path)
            config = load_config(config_path)
            sorted_results = execute_arima(
                frame_interesting_columns, config, column_to_predict, split_percent, n_future=n_future)
            os.system('rm ' + config_path)
        else:
            config = evaluate_models(
                frame_interesting_columns, column_to_predict, split_percent)

            save_config(config_path, config)

            sorted_results = execute_arima(
                frame_interesting_columns, config, column_to_predict, split_percent, n_future=n_future)

        avg_mae = np.mean(np.array(list(map(lambda x:x[1][0], sorted_results))))
        avg_mape = np.mean(np.array(list(map(lambda x:x[1][1], sorted_results))))

        summary = '|' + column_to_predict + '| seq len ' + str(n_future) + '| mae: ' + str(avg_mae) + '| mape: ' + str(avg_mape) + '| best config: ' + str(config)

        print(summary)
        summaries.append(summary)

        if n_future > 2:
            plot_best_pred(sorted_results, column_to_predict)
        else:
            print(('Pred: ', sorted_results[0][1][-1]['pred'].values))
            print(('Test: ', sorted_results[0][1][-1]['y_test'].values))

print(summaries)



KeyboardInterrupt: ignored

##Risultati

###Deceduti
|    | LSTM                           | XGBoost                        | ARIMA                          | Informer                       | NeuralProphet                       |
|----|--------------------------------|--------------------------------|--------------------------------|--------------------------------|--------------------------------|
| 1  | Avg MAE: 3.18  <br />Best Slot: 0.03 | Avg MAE: 2.78  <br />Best Slot: 0.02 | Avg MAE: **2.42**  <br />Best Slot: 0.01 | Avg MAE: **2.26**  <br />Best Slot: 0.008| Avg MAE: 3.18 |
| 2  | Avg MAE: 4.03  <br />Best Slot: 0.33 | Avg MAE: 2.97  <br />Best Slot: 0.11 | Avg MAE: **2.45**  <br />Best Slot: 0.44 | Avg MAE: **2.35**  <br />Best Slot: 0.08 | Avg MAE: 6.94 |
| 7  | Avg MAE: **2.51**  <br />Best Slot: 1.01 | Avg MAE: 3.30  <br />Best Slot: 1.19 | Avg MAE: 2.53  <br />Best Slot: 1.06 | Avg MAE: 2.70  <br />Best Slot: 1.20 | Avg MAE: 6.73  |
| 14 | Avg MAE: 6.22  <br />Best Slot: 3.49 | Avg MAE: 3.31  <br />Best Slot: 1.62 | Avg MAE: **2.50**  <br />Best Slot: 1.21 | Avg MAE: 4.02  <br />Best Slot: 1.74 | Avg MAE: 2.75   |


###Terapia Intensiva
|    | LSTM                           | XGBoost                        | ARIMA                          | Informer                       | NeuralProphet                       |
|----|--------------------------------|--------------------------------|--------------------------------|--------------------------------|--------------------------------|
| 1  | Avg MAE: 4.36  <br />Best Slot: 0.03 | Avg MAE: 3.21  <br />Best Slot: 0.002 | Avg MAE: **2.31**  <br />Best Slot: 0.02 | Avg MAE: 13.61  <br />Best Slot: 1.53| Avg MAE: 16.22  |
| 2  | Avg MAE: 5.83  <br />Best Slot: 1.08 | Avg MAE: 4.55  <br />Best Slot: 0.47 | Avg MAE: **2.87**  <br />Best Slot: 0.05 | Avg MAE: 5.17  <br />Best Slot: 0.54 | Avg MAE: 3.28   |
| 7  | Avg MAE: 11.09  <br />Best Slot: 1.95 | Avg MAE: 7.03  <br />Best Slot: 1.89 | Avg MAE: 4.73  <br />Best Slot: 1.05 | Avg MAE: 8.53  <br />Best Slot: 2.88 | Avg MAE: **4.57**  |
| 14 | Avg MAE: 13.17  <br />Best Slot: 5.95 | Avg MAE: 9.37  <br />Best Slot: 2.96 | Avg MAE: **6.86**  <br />Best Slot: 1.22 | Avg MAE: 15.38  <br />Best Slot: 8.00| Avg MAE: 19.25   |

###Nuovi Positivi
|    | LSTM                              | XGBoost                           | ARIMA                             | Informer                       | NeuralProphet                       |
|----|-----------------------------------|-----------------------------------|-----------------------------------|--------------------------------|--------------------------------|
| 1  | Avg MAE: 118.94  <br />Best Slot: 0.03  | Avg MAE: 74.04  <br />Best Slot: 0.56   | Avg MAE: **57.19**  <br />Best Slot: 0.07   | Avg MAE: 118.74  <br />Best Slot: 1.82 | Avg MAE: 89.37  |
| 2  | Avg MAE: 113.78  <br />Best Slot: 6.82  | Avg MAE: 88.25  <br />Best Slot: 6.17   | Avg MAE: **65.80**  <br />Best Slot: 3.98   | Avg MAE: 90.73   <br />Best Slot: 4.62   | Avg MAE: 79.94  |
| 7  | Avg MAE: 132.39  <br />Best Slot: 33.74 | Avg MAE: 113.55  <br />Best Slot: 28.43 | Avg MAE: **85.05**  <br />Best Slot: 16.71  | Avg MAE: 162.37  <br />Best Slot: 41.94  | Avg MAE: 220.26  |
| 14 | Avg MAE: 156.96  <br />Best Slot: 48.71 | Avg MAE: 169.45  <br />Best Slot: 27.14 | Avg MAE: **114.03**  <br />Best Slot: 22.81 | Avg MAE: 529.82  <br />Best Slot: 351.47 | Avg MAE: 138.67   |

ARIMA mantiene un errore di previsione inferiore rispetto agli altri modelli in tutti gli scenari tranne per:
- Orizzonte deceduti di 7 giorni in cui LSTM ha MAE inferiore
- Orizzonte 1 e 2 giorni di deceduti in cui Informer ha un errore inferiore
- Terapia intensiva con orizzonte 7 giorni in cui l'errore inferiore è con NerualProphet.
